In [1]:
from brightway2 import *
import brightway2 as bw
import bw2data as bd
import bw2calc as bc
import pandas as pd
import numpy as np
import tabulate
import os

In [2]:
projects.set_current("ecoinvent38_import")
db38 = Database("cutoff38")

In [3]:
bw2setup()

In [4]:
def print_recursive_calculation(activity, lcia_method, df_output, lca_obj=None, total_score=None, amount=1, mother='root', self='no', level=0, max_level=3, cutoff=1e-2):
    if lca_obj is None:
        lca_obj = bc.LCA({activity: amount}, lcia_method)
        lca_obj.lci()
        lca_obj.lcia()
        total_score = lca_obj.score
    elif total_score is None:
        raise ValueError
    else:
        lca_obj.redo_lcia({activity: amount})
        if abs(lca_obj.score) <= abs(total_score * cutoff):
            return df_output
        
    d_em = lca_obj.characterized_inventory[:, lca_obj.activity_dict.get(activity.key)].sum()
    tmp = pd.DataFrame(
        [[level, str(activity), mother, self, lca_obj.score / total_score, lca_obj.score, d_em/total_score, d_em]], 
        columns=['level', 'process_name', 'mother_uuid', 'self_uuid', 'contri_pct', 'emission_kg', 'direct_contri_pct', 'direct_emissions_kg']
    )
    df_output = pd.concat([df_output, tmp], ignore_index=True)
    
    if level < max_level:
        for exc in activity.technosphere():
            df_output = print_recursive_calculation(
                activity = exc.input,
                lcia_method=lcia_method,
                df_output = df_output,
                lca_obj=lca_obj,
                total_score=total_score,
                amount=amount * exc['amount'],
                mother=dict(exc)['output'][1],
                self=dict(exc)['input'][1],
                level=level + 1,
                max_level=max_level, 
                cutoff=cutoff
            )
    
    df_output.at[0, 'self_uuid'] = dict(activity)['code']
    
    return df_output

In [5]:
GWP_method = [m for m in bw.methods if 'TRACI' in str(m) and 'environmental impact' in str(m) and 'global warming' in str(m)][0]
GWP_method

In [6]:
# use number to replace uuid, easy for tracking previous node
def conver_node(df):
    keys = list(set(df['mother_uuid'][1:].tolist()))
    values = range(len(keys))
    mapping = {k: v for k, v in zip(keys, values)}
    mapping['root'] = 'root'
    
    ls_uuid = df['mother_uuid'].tolist()
    converted_list = [mapping[item] for item in ls_uuid]
    
    return converted_list

In [7]:
# for i in range(sc_lca.shape[0]):
#     df_output = pd.DataFrame()
#     uuid = sc_lca.at[i,'uuid']
#     process = db38.get(uuid)
#     df_output = print_recursive_calculation(process, GWP_method, df_output, max_level=5, cutoff=0.01)
    
#     df_output.to_csv(f'new_disagg_proc{i}.csv')

## Example - Ammonia

In [8]:
NH3_output = pd.DataFrame()
NH3 = db38.search('ammonia production, steam reforming, liquid')[1]
NH3_id = dict(NH3)['code']
NH3_output = print_recursive_calculation(NH3, GWP_method, NH3_output, max_level=10, cutoff=0.001)